# Here we are using the model 'Synthea' to generate patients as a DataSet so we can work with it

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date


## Selecting the tables that we want to work on.

## which is:
### - allergies
### - medications
### - patients


In [2]:
df_allergies = pd.read_csv("/Users/aziz/code/raynasser/MediSure/MediSure/generative_models/Synthea/output/csv/allergies.csv")
df_medications = pd.read_csv("/Users/aziz/code/raynasser/MediSure/MediSure/generative_models/Synthea/output/csv/medications.csv")
df_patients = pd.read_csv("/Users/aziz/code/raynasser/MediSure/MediSure/generative_models/Synthea/output/csv/patients.csv")

  ##     Adding the 'AGE' column from the 'BIRTHDAY' column.

In [3]:
df_patients["BIRTHDATE"] = pd.to_datetime(df_patients['BIRTHDATE'])
df_patients['today'] = pd.to_datetime(dt.date.today())
df_patients['Age'] = (df_patients['today'] - df_patients['BIRTHDATE']).dt.days/365
df_patients['Age'] = df_patients['Age'].apply(lambda x: int(x))

df_patients.drop(columns={'today'}, inplace=True)

## renaming Id to PATIENT, in patients table

In [4]:
df_patients.rename(columns={'Id': 'PATIENT'}, inplace=True)


## Drop useless features from the columns

In [5]:
df_allergies = df_allergies.drop(columns=['START','STOP','ENCOUNTER','REACTION1','DESCRIPTION1','SEVERITY1','REACTION2','DESCRIPTION2','SEVERITY2'])
df_medications.drop(columns = ['REASONDESCRIPTION','CODE','START', 'STOP','PAYER','BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE'], inplace = True)
df_patients.drop(columns = ['BIRTHDATE','DEATHDATE','SSN','DRIVERS','PASSPORT','PREFIX','SUFFIX','MAIDEN','MARITAL','RACE',
       'ETHNICITY','FIPS', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES',
       'HEALTHCARE_COVERAGE', 'INCOME','STATE','BIRTHPLACE','STATE','COUNTY'], inplace = True)

### grouping the allergies table by patient column and separating each allergy for a specefic patient in a new column

In [6]:
# Group by 'PATIENT' and create a list of allergies for each patient
patient_allergies = df_allergies.groupby('PATIENT')['DESCRIPTION'].apply(list).reset_index()

# Expand the list of allergies into separate columns
patient_allergies_expanded = pd.DataFrame(patient_allergies['DESCRIPTION'].tolist(), index=patient_allergies['PATIENT']).reset_index()

# Dynamically rename columns based on the maximum number of allergies
max_allergies = patient_allergies_expanded.shape[1] - 1  # Subtract 1 for the 'PATIENT' column
new_column_names = ['PATIENT'] + [f'Allergy {i+1}' for i in range(max_allergies)]
patient_allergies_expanded.columns = new_column_names

In [7]:
patient_allergies_expanded.drop(columns=['Allergy 1'], inplace=True)

# Reassign column names based on the new number of allergies
# Get the number of remaining allergy columns
max_allergies = patient_allergies_expanded.shape[1] - 1  # Subtract 1 for the 'PATIENT' column

# Create new column names
new_column_names = ['PATIENT'] + [f'Allergy {i+1}' for i in range(max_allergies)]
patient_allergies_expanded.columns = new_column_names


### grouping the medication table by patient column and separating each medication for a specefic patient in a new column

In [8]:
# Group by 'PATIENT' and create a list of medications for each patient
patient_medications = df_medications.groupby('PATIENT')['DESCRIPTION'].apply(list).reset_index()

# Find the maximum number of medications any single patient has
max_medications = patient_medications['DESCRIPTION'].apply(len).max()

# Expand the list of medications into separate columns
patient_medications_expanded = pd.DataFrame(patient_medications['DESCRIPTION'].tolist(), index=patient_medications['PATIENT']).reset_index()

# Dynamically rename columns based on the number of medications
new_column_names = ['PATIENT'] + [f'Medication {i+1}' for i in range(max_medications)]
patient_medications_expanded.columns = new_column_names

### saving the medications in a csv file

In [9]:
medication_columns = [col for col in patient_medications_expanded.columns if col.startswith('Medication')]

# Concatenate all medications into a single list and drop duplicates
unique_medications = pd.unique(patient_medications_expanded[medication_columns].values.ravel('K'))

# Convert the result into a list (optional)
unique_medications_list = unique_medications.tolist()

# Display the unique medications
print(unique_medications_list)

medications_df = pd.DataFrame(unique_medications_list, columns=['Unique Medications'])

# Save the DataFrame to a CSV file
medications_df.to_csv('unique_medications.csv', index=False)


['Naproxen sodium 220 MG Oral Tablet', 'Vitamin B12 5 MG/ML Injectable Solution', 'Chlorpheniramine Maleate 2 MG/ML Oral Solution', 'sodium fluoride 0.0272 MG/MG Oral Gel', 'Simvastatin 10 MG Oral Tablet', 'Hydrochlorothiazide 25 MG Oral Tablet', 'Ibuprofen 100 MG Oral Tablet', 'insulin isophane  human 70 UNT/ML / insulin  regular  human 30 UNT/ML Injectable Suspension [Humulin]', 'Acetaminophen 325 MG Oral Tablet', 'Loratadine 5 MG Chewable Tablet', 'diphenhydrAMINE Hydrochloride 25 MG Oral Tablet', 'Hydrocortisone 10 MG/ML Topical Cream', 'cetirizine hydrochloride 5 MG Oral Tablet', '1 ML medroxyprogesterone acetate 150 MG/ML Injection', 'Natazia 28 Day Pack', 'Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet', 'Kyleena 19.5 MG Intrauterine System', 'ferrous sulfate 325 MG Oral Tablet', 'Clopidogrel 75 MG Oral Tablet', 'Alteplase 100 MG Injection', 'carbamazepine 20 MG/ML Oral Suspension [Tegretol]', 'aspirin 81 MG Oral Tablet', 'lisinopril 10 MG Oral Tablet', 'atomoxetine 100 MG 

# merging the tables into one table to have a complete profile of the patient

In [10]:
# Merge df_patients with patient_allergies_expanded
merged_allergies = pd.merge(df_patients, patient_allergies_expanded, on='PATIENT', how='left')

# Merge the result with patient_medications_expanded
final_merged = pd.merge(merged_allergies, patient_medications_expanded, on='PATIENT', how='left')


## droping any patient that doesnt take allergy nor medication 

In [11]:
# Drop patients with no allergies and no medications
# Get the list of allergy and medication columns
allergy_columns = [f'Allergy {i}' for i in range(1, 13)]  # Assuming there are 12 allergy columns
medication_columns = [f'Medication {i}' for i in range(1, 2020)]  # Assuming there are 2019 medication columns

# Filter the DataFrame to keep only patients with at least one allergy or medication
profiles = final_merged.dropna(subset=allergy_columns + medication_columns, how='all')


# free testing 👇🏻

In [37]:
profiles.iloc[11][27]

# profiles.columns[23]

/var/folders/lx/dlcmglnd5514lttspw0c50x80000gp/T/ipykernel_15988/2091724463.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profiles.iloc[11][27]


'Simvastatin 10 MG Oral Tablet'

In [38]:
profiles.to_csv('profiles.csv', index=False)

In [92]:
patient_medications_expanded[patient_medications_expanded['Medication 2019'] == 'amLODIPine 2.5 MG Oral Tablet']

,PATIENT,Medication 1,Medication 2,Medication 3,Medication 4,Medication 5,Medication 6,Medication 7,Medication 8,Medication 9,...,Medication 2010,Medication 2011,Medication 2012,Medication 2013,Medication 2014,Medication 2015,Medication 2016,Medication 2017,Medication 2018,Medication 2019
1810,ce452a7b-2543-e768-c6bb-cabddb6665a3,24 HR metoprolol succinate 100 MG Extended Rel...,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray,insulin isophane human 70 UNT/ML / insulin r...,Hydrochlorothiazide 25 MG Oral Tablet,24 HR tacrolimus 1 MG Extended Release Oral Ta...,lisinopril 10 MG Oral Tablet,amLODIPine 2.5 MG Oral Tablet,insulin isophane human 70 UNT/ML / insulin r...,Hydrochlorothiazide 25 MG Oral Tablet,...,insulin isophane human 70 UNT/ML / insulin r...,Hydrochlorothiazide 25 MG Oral Tablet,24 HR tacrolimus 1 MG Extended Release Oral Ta...,lisinopril 10 MG Oral Tablet,amLODIPine 2.5 MG Oral Tablet,insulin isophane human 70 UNT/ML / insulin r...,Hydrochlorothiazide 25 MG Oral Tablet,24 HR tacrolimus 1 MG Extended Release Oral Ta...,lisinopril 10 MG Oral Tablet,amLODIPine 2.5 MG Oral Tablet


In [9]:
df_patients

,Id,FIRST,MIDDLE,LAST,GENDER,ADDRESS,CITY,Age
0,a3701e6e-6d11-32cd-a4e6-8515efcb34f2,Micah422,Kenda873,Gutkowski940,F,559 Johnston Bay Unit 85,Boston,12
1,e7d7cef3-0f41-afc5-0c72-2da66c33cdc5,Juan88,Lucien408,Gutmann970,M,517 Mante Mews,Newton,39
2,2a91b214-afb8-339f-2bc5-b80b679e5fd2,Irwin931,Graig740,Sipes176,M,962 Crona Course,Boston,51
3,d8f6e82b-e0c9-46bb-a3f0-d1e9d4da045d,Jack927,NaN,Collins926,M,268 Pacocha Esplanade,Reading,52
4,3af2a564-3bfd-7c9a-3d44-575e6e0cf3b3,Yuri475,Armandina361,Nienow652,F,303 Dietrich Ville,Belmont,8
...,...,...,...,...,...,...,...,...
2338,a3bb83be-b30f-8697-b4c7-fb190c597464,Melissa844,Lasonya941,Kunze215,F,900 Trantow Course,Lowell,110
2339,f11123a1-7513-4826-bea5-441ede68fe92,Garret233,Gilbert263,Purdy2,M,329 O'Conner Junction Unit 54,North Plymouth,47
2340,c9b1bda9-3c61-8618-c55a-4565c4ff9469,Kassandra256,Aletha771,Roberts511,F,562 Schneider Way,Lowell,110
2341,988452fe-6908-ad38-8e19-302a3bb6c72c,Wesley533,Modesta635,Smitham825,F,485 Lebsack Track,Haverhill,84


In [23]:
profiles

,PATIENT,FIRST,MIDDLE,LAST,GENDER,ADDRESS,CITY,Age,Allergy 1,Allergy 2,...,Medication 2010,Medication 2011,Medication 2012,Medication 2013,Medication 2014,Medication 2015,Medication 2016,Medication 2017,Medication 2018,Medication 2019
0,a3701e6e-6d11-32cd-a4e6-8515efcb34f2,Micah422,Kenda873,Gutkowski940,F,559 Johnston Bay Unit 85,Boston,12,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,e7d7cef3-0f41-afc5-0c72-2da66c33cdc5,Juan88,Lucien408,Gutmann970,M,517 Mante Mews,Newton,39,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,2a91b214-afb8-339f-2bc5-b80b679e5fd2,Irwin931,Graig740,Sipes176,M,962 Crona Course,Boston,51,Animal dander (substance),Sulfamethoxazole / Trimethoprim,...,None,None,None,None,None,None,None,None,None,None
3,d8f6e82b-e0c9-46bb-a3f0-d1e9d4da045d,Jack927,NaN,Collins926,M,268 Pacocha Esplanade,Reading,52,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,3af2a564-3bfd-7c9a-3d44-575e6e0cf3b3,Yuri475,Armandina361,Nienow652,F,303 Dietrich Ville,Belmont,8,Mold (organism),Animal dander (substance),...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,a3bb83be-b30f-8697-b4c7-fb190c597464,Melissa844,Lasonya941,Kunze215,F,900 Trantow Course,Lowell,110,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2339,f11123a1-7513-4826-bea5-441ede68fe92,Garret233,Gilbert263,Purdy2,M,329 O'Conner Junction Unit 54,North Plymouth,47,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2340,c9b1bda9-3c61-8618-c55a-4565c4ff9469,Kassandra256,Aletha771,Roberts511,F,562 Schneider Way,Lowell,110,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2341,988452fe-6908-ad38-8e19-302a3bb6c72c,Wesley533,Modesta635,Smitham825,F,485 Lebsack Track,Haverhill,84,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
